<a href="https://colab.research.google.com/github/JefNtungila/Hostels-in-SEA/blob/master/Hostels_in_the_Philippines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hostels in the Philippines

In [0]:
from bs4 import BeautifulSoup
import requests
import math
import pandas as pd

##Loading in the Cities

In [0]:
country = 'Philippines'

In [0]:


result = requests.get(f"https://www.hostelworld.com/hostels/{country}")

In [0]:
src = result.content
soup = BeautifulSoup(src, 'lxml')
soup.prettify();

In [25]:
locations = ['topcities', 'otherlocations']


list_of_cities = []

for location in locations:
  top_cities_finder = soup.find_all('div', class_=(f"{location}")) #find html classes
  list_of_cities.append(top_cities_finder)

#for every item in the sublist and for every sublist in the list of cities - get all the items
flat_list_of_cities = [item for sublist in list_of_cities for item in sublist]


cities = []

for city in flat_list_of_cities:
  city = city.find_all('a') #find articles in html classes
  cities.append(city)
  
final_flat_list_of_cities = [item for sublist in cities for item in sublist]

city_names = []

for city in final_flat_list_of_cities:
  city_name = city.text
  city_names.append(city_name)

print( len(list_of_cities), len(flat_list_of_cities) , len(cities) , len(final_flat_list_of_cities), len(city_names))

2 3 3 40 40


In [20]:
city_names

['Cebu',
 'Manila',
 'El Nido',
 'Boracay Island',
 'Coron',
 'Puerto Princesa City',
 'Siargao Island',
 'Panglao Island',
 'Malapascua Island',
 'Bohol Island',
 'Bohol',
 'Palawan',
 'Baguio',
 'Baler',
 'Banaue',
 'Batuan',
 'Bohol Island',
 'Boracay Island',
 'Carabao Island',
 'Cebu',
 'Coron',
 'Davao City',
 'Dumaguete',
 'El Nido',
 'Kalibo',
 'Legazpi City',
 'Malapascua Island',
 'Manila',
 'Mati City',
 'Panglao Island',
 'Puerto Galera',
 'Puerto Princesa City',
 'San Felipe',
 'San Jose Negros Oriental',
 'San Juan La Union',
 'San Vicente',
 'Siargao Island',
 'Siquijor',
 'Tablas Island',
 'Tagaytay']

##Getting hostel names

In [0]:
date_from = '2019-07-09'
date_to = {'2019-07-12'}

In [0]:
hostel_names = []

for city in city_names:
  try: 
    result1 = requests.get(f"https://www.hostelworld.com/findabed.php/ChosenCity.{city}/ChosenCountry.{country}&date_from={date_from}&date_to={date_to}")
    src1 = result1.content
    soup1 = BeautifulSoup(src1, 'lxml')
    soup1.prettify();

    number_of_hostels = soup1.find_all('div', class_ = 'row fabfooter')
    number_of_hostels = str(number_of_hostels[0]).split(f'''<span> hostels in {city}''')
    number_of_hostels = str(number_of_hostels).split('We have ')
    number_of_hostels = number_of_hostels[1].split(''' \', \'</span>\\n\\n''')
    number_of_hostels = int(number_of_hostels[0])#finding how many hostels are in the city
    webpages = math.ceil(number_of_hostels/30) #finding how many pages we need to cover
    webpages

    page_number = webpages+1

    for page in range (1, page_number):
      print (page)
      result2 = requests.get(f"https://www.hostelworld.com/findabed.php/ChosenCity.{city}/ChosenCountry.{country}&date_from={date_from}&date_to={date_to}?page={page}")
      src2 = result2.content
      soup2 = BeautifulSoup(src2, 'lxml')
      soup2.prettify();

      hostel_container = soup2.find_all('div', class_ = 'fabresult rounded clearfix hwta-property')
      for item in hostel_container:
        item = item.find('a').text
        hostel_names.append(item)
    #     print(item)
  except:
      print('excuse me no hostels found')

print(len(hostel_names))


In [124]:
df = pd.DataFrame(hostel_names, columns = ['Hostel Names'])
df.head(20)

,Hostel Names
0,Noordzee Hostel
1,HappyNest Hostel
2,Murals Hostel and Cafe
3,Teofel Hostel
4,Sharky Hostel Oslob
5,Mad Monkey Cebu City
6,Chief Mau Moalboal Cebu
7,Moalboal Backpacker Lodge
8,Casa Bonita Inn Oslob
9,Le Village Guesthouse Hostel & Bar


In [125]:
df.tail(20)

,Hostel Names
424,Avocado Tree Hostel
425,Footprints Homestay
426,Salty Nomads
427,Lampara Siargao
428,Paglaom Hostel
429,Secret Spot Siargao
430,Kawili Hostel
431,Coco Pod Hostel
432,Sunlit Hostel
433,Kokai Resort
